<a href="https://colab.research.google.com/drive/1HLxEVsGfXvEm73Rszwd-ucUW6wkQtE9e?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Generative Agents

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass
from datetime import datetime

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [6]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [8]:
genai.configure(api_key=API_KEY)

In [9]:
class GenerativeAgent:
    def __init__(self, name):
        self.model = genai.GenerativeModel("gemini-2.0-flash")
        self.name = name
        self.memory_stream = []  # All observations
        self.reflections = []    # High-level insights
        self.environment = {}    # Current environment state

    def observe(self, observation):
        """Record observation in memory stream"""
        memory = {
            "time": datetime.now().strftime("%H:%M"),
            "type": "observation",
            "content": observation
        }
        self.memory_stream.append(memory)
        print(f"👁️  {self.name} observed: {observation}")

    def reflect(self):
        """Generate insights from recent memories"""
        if len(self.memory_stream) < 3:
            return

        recent = "\n".join([m["content"] for m in self.memory_stream[-5:]])

        prompt = f"""Based on these observations, generate 2-3 high-level insights:

        {recent}

        Insights:"""

        response = self.model.generate_content(prompt).text

        reflection = {
            "time": datetime.now().strftime("%H:%M"),
            "type": "reflection",
            "content": response.strip()
        }
        self.reflections.append(reflection)
        print(f"💭 {self.name} reflected: {response.strip()}\n")

    def plan(self, goal):
        """Create action plan based on memories and environment"""
        memories = "\n".join([m["content"] for m in self.memory_stream[-3:]])
        insights = "\n".join([r["content"] for r in self.reflections[-2:]])
        env = str(self.environment)

        prompt = f"""You are {self.name}. Create a short action plan.

        Goal: {goal}

        Recent memories:
        {memories}

        Insights:
        {insights if insights else "None yet"}

        Environment:
        {env}

        Plan (3-5 actions):"""

        plan = self.model.generate_content(prompt).text
        print(f"📋 {self.name}'s plan:\n{plan}\n")
        return plan.strip()

    def act(self, action, environment):
        """Take action and update environment"""
        print(f"⚡ {self.name} action: {action}")

        # Simulate action effect on environment
        prompt = f"""An agent performed this action: {action}

        Current environment:
        {environment}

        How does the environment change? Respond with key-value pairs.
        Format: key: new_value

        Changes:"""

        response = self.model.generate_content(prompt).text

        # Parse environment changes
        for line in response.split("\n"):
            if ":" in line:
                key, val = line.split(":", 1)
                environment[key.strip()] = val.strip()

        print(f"🌍 Environment updated: {environment}\n")
        return environment

    def run_cycle(self, goal, steps=5):
        """Run observe-reflect-plan-act cycle"""
        print(f"\n{'='*60}")
        print(f"🎯 {self.name}: {goal}")
        print(f"{'='*60}\n")

        for i in range(steps):
            print(f"--- Cycle {i+1} ---\n")

            # Observe environment
            self.observe(f"Environment state: {self.environment}")

            # Reflect periodically
            if i > 0 and i % 2 == 0:
                self.reflect()

            # Plan next action
            plan = self.plan(goal)

            # Extract first action from plan
            action = plan.split("\n")[0].strip()

            # Act and update environment
            self.environment = self.act(action, self.environment)

            # Record action in memory
            self.memory_stream.append({
                "time": datetime.now().strftime("%H:%M"),
                "type": "action",
                "content": action
            })

        print(f"✅ Goal completed: {goal}\n")
        self.show_summary()

    def show_summary(self):
        """Show agent's memory and insights"""
        print(f"{'='*60}")
        print(f"📊 {self.name}'s Summary")
        print(f"{'='*60}")
        print(f"Memories: {len(self.memory_stream)}")
        print(f"Reflections: {len(self.reflections)}")
        print(f"Final environment: {self.environment}")

        if self.reflections:
            print(f"\nKey insights:")
            for r in self.reflections[-2:]:
                print(f"  • {r['content'][:80]}...")

In [10]:
# Example 1: Research Assistant Agent
print("="*60)
print("EXAMPLE 1: Research Assistant Agent")
print("="*60)

researcher = GenerativeAgent("ResearchBot")
researcher.environment = {
    "papers_read": 0,
    "notes": "empty",
    "draft": "not started"
}

researcher.run_cycle("Research AI agents and write summary", steps=4)


# Example 2: Customer Service Agent
print("\n" + "="*60)
print("EXAMPLE 2: Customer Service Agent")
print("="*60)

cs_agent = GenerativeAgent("SupportBot")
cs_agent.environment = {
    "tickets": 5,
    "customer_satisfaction": "unknown",
    "resolved": 0
}

cs_agent.run_cycle("Handle customer support tickets", steps=3)


# Example 3: Multi-agent simulation
print("\n" + "="*60)
print("EXAMPLE 3: Multi-Agent Collaboration")
print("="*60)

manager = GenerativeAgent("Manager")
developer = GenerativeAgent("Developer")

# Shared environment
shared_env = {
    "project_status": "not started",
    "tasks": 10,
    "completed": 0
}

manager.environment = shared_env.copy()
developer.environment = shared_env.copy()

print("--- Manager's turn ---")
manager.observe("New project assigned with 10 tasks")
plan = manager.plan("Organize project and assign tasks")
action = plan.split("\n")[0]
shared_env = manager.act(action, shared_env)

print("\n--- Developer's turn ---")
developer.environment = shared_env.copy()
developer.observe(f"Manager assigned tasks: {shared_env}")
dev_plan = developer.plan("Complete assigned tasks")
dev_action = dev_plan.split("\n")[0]
shared_env = developer.act(dev_action, shared_env)

print(f"\n🌍 Final shared environment: {shared_env}")

EXAMPLE 1: Research Assistant Agent

🎯 ResearchBot: Research AI agents and write summary

--- Cycle 1 ---

👁️  ResearchBot observed: Environment state: {'papers_read': 0, 'notes': 'empty', 'draft': 'not started'}
📋 ResearchBot's plan:
Okay, here's a short action plan to start researching AI agents and writing a summary:

1.  **Identify Relevant Keywords:** Brainstorm and list keywords related to AI agents (e.g., "autonomous agents," "intelligent agents," "multi-agent systems," "AI agent architectures," "reinforcement learning agents," "cognitive agents").
2.  **Search for Initial Papers:** Use a search engine (e.g., Google Scholar, arXiv) to find 2-3 highly cited or recent survey papers/review articles on AI agents using the keywords.
3.  **Read and Take Notes on Paper 1:**  Read the abstract and introduction of the first paper to understand the scope. If relevant, skim the rest of the paper, taking brief notes on key concepts, architectures, and applications.
4. **Add Notes:** Save th